In [ ]:
import warnings
warnings.filterwarnings("ignore")
import gc
import os
import netCDF4 as nc4
import numpy as np
import numpy.ma as ma
import pandas as pd
import geopy as gpy
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely import Polygon, box

In [ ]:
withoffset = nc4.Dataset('/scratch/07174/soelem/test_jason/fort.63.nc')
withoutoffset = nc4.Dataset('/work2/07174/soelem/data/fort.63.nc')

In [ ]:
lat = withoffset.variables['y'][:].data
lon = withoffset.variables['x'][:].data

In [ ]:
start_date = pd.Timestamp('2023-07-16 00:00:00')
end_date = pd.Timestamp('2023-10-31 23:00:00')
time = pd.date_range(start_date, end_date, freq='H')

In [ ]:
# Define the directory where you want to save the files
output_dir = '/scratch/07174/soelem/results'

#Map Extent Bounds
lat1,lat2 = 7, 50
lon1,lon2 = -100, -58
cmap =  mpl.cm.get_cmap('seismic')
for i in range(len(time)):
    #Basic Plot parameters
    ssh_with = withoffset.variables['zeta'][i,:].data
    ssh_without = withoutoffset.variables['zeta'][i,:].data
    diff = ssh_with - ssh_without
    ts = pd.to_datetime(str(time[i]))
    fig  = plt.figure(figsize=(10,6))
    proj = ccrs.PlateCarree()
    ax = plt.axes(projection = proj)
    ax.set_extent([lon1, lon2, lat1, lat2], crs=proj)
    cmap =  mpl.cm.get_cmap('seismic')
    levels = np.arange(-0.9, 0.9, 0.01)
    triang = tri.Triangulation(lon, lat, triangles=withoffset.variables['element'][:,:]-1)
    if ma.isMaskedArray(diff):  # Check if diff is a masked array
        if diff.mask.any():  # Then check if any element is masked
            point_mask_indices = np.where(diff.mask)
            tri_mask = np.any(np.in1d(withoffset.variables['element'][:,:]-1, point_mask_indices).reshape(-1, 3), axis=1)
            triang.set_mask(tri_mask)
    diff_array = np.array(diff.data)  # Convert memoryview to numpy array
    diff_array[diff_array<-999] = np.nan  # Now you can do the comparison and assignment
    if diff.size == 0 or np.all(np.isnan(diff)):  # Check if data is empty or all NaNs
        print(f"Skipping iteration {i} due to invalid data.")
        continue  # Skip this iteration if data is not valid
    cf = ax.tricontourf(triang, diff.data, transform = proj, cmap = cmap, levels=levels,extend='both')
    ax.coastlines(resolution='10m', linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgrey')
    cbar = fig.colorbar(cf)
    cbar.set_label('WL Difference (m)', size=25)
    cbar.ax.tick_params(labelsize=15)
    gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
    
    # Customize the gridlines
    gl.xlabels_top = False  # Hide the top x-axis labels
    gl.ylabels_right = False  # Hide the right y-axis labels
    gl.xlabel_style = {'size': 10}  # Set the x-axis label font size
    gl.ylabel_style = {'size': 10, 'rotation': 90}  # Set the y-axis label font size and rotation
    #norm = plt.Normalize(min, max)
    #cbar = plt.colorbar(norm=norm, cmap=cmap)
    # plt.title ('2014-07-01 10:00', size=15)
    savename = os.path.join(output_dir, ts.strftime('%Y-%m-%d %H.%M'))
    plt.title(savename, size=15)
    title = str(f'{savename}.png')
    try:
        fig.savefig(title, dpi=100, bbox_inches='tight')
    except ValueError as e:
        print(f"An error occurred while saving the figure in iteration {i}: {e}")
        continue  # Skip the iteration if an error occurred during saving
    
    plt.close('all')  # Close all figures to ensure memory is freed
    del ssh_with, ssh_without, diff, cf, fig, ax, triang  # Delete large variables explicitly
    gc.collect()  # Suggest to the Python garbage collector to free up memory

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import matplotlib.tri as tri
import cartopy.feature as cfeature

# Assuming you have already loaded your datasets and extracted necessary variables
# Specify the timestep you're interested in
timestep = 400  # Replace with your specific timestep

# Define the levels for contouring
levels = np.arange(-0.9, 0.9, 0.01)  # Adjust as needed

# Create the figure and subplots
fig, axs = plt.subplots(1, 2, figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()}, gridspec_kw={'wspace': 0.05})

# Plot for 'withoutoffset' dataset on the left
ssh_without = withoutoffset.variables['zeta'][timestep, :].data
triang_without = tri.Triangulation(lon, lat, triangles=withoutoffset.variables['element'][:, :] - 1)
cf_without = axs[0].tricontourf(triang_without, ssh_without, levels=levels, cmap='viridis', extend='both')
axs[0].coastlines()
axs[0].add_feature(cfeature.LAND, facecolor='darkgrey')
gl_without = axs[0].gridlines(draw_labels=True, xlocs=np.arange(-180, 181, 10), ylocs=np.arange(-90, 91, 10))
gl_without.ylabels_right = False
gl_without.xlabels_top = False
gl_without.xlabel_style = {'size': 12}
gl_without.ylabel_style = {'rotation': 90, 'size': 12}

# Plot for 'withoffset' dataset on the right
ssh_with = withoffset.variables['zeta'][timestep, :].data
triang_with = tri.Triangulation(lon, lat, triangles=withoffset.variables['element'][:, :] - 1)
cf_with = axs[1].tricontourf(triang_with, ssh_with, levels=levels, cmap='viridis', extend='both')
axs[1].coastlines()
axs[1].add_feature(cfeature.LAND, facecolor='darkgrey')
gl_with = axs[1].gridlines(draw_labels=True, xlocs=np.arange(-180, 181, 10), ylocs=np.arange(-90, 91, 10))
gl_with.ylabels_left = False
gl_with.ylabels_right = False
gl_with.xlabels_top = False
gl_without.xlabel_style = {'size': 12}

# Position the colorbar outside and below the plot boundaries
cbar = fig.colorbar(cf_with, ax=axs.ravel().tolist(), orientation='horizontal', pad=0.05, fraction=0.05)
cbar.set_label('Water Level (m)', size=20)

# Adjust layout
plt.tight_layout()

# Save the figure
plt.savefig("water_levels_comparison.png", dpi=600)

# Show the plot
plt.show()